## Importing Libraries

In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
import pickle as pkl

In [76]:
df = pd.read_pickle('df_train_api.pk')

In [77]:
df.head()

,groups,type,data,label,coords
270,ACME VISUAL SYSTEMS,"(acm visual system, text,text,text, 2337, 1653)",14754,0,"[511.0, 143.0, 1217.0, 189.0, 864.0, 166.0]"
271,TAX INVOICE,"(tax invoic, text,text, 2337, 1653)",14754,0,"[717.0, 264.0, 877.0, 284.0, 797.0, 274.0]"
272,ACME VISUAL SYSTEMS,"(acm visual system, text,text,text, 2337, 1653)",14754,0,"[166.0, 351.0, 435.0, 369.0, 300.5, 360.0]"
273,"808, JANTA FLAT, GTB ENCLAVE, NANAD NAGARI","(number janta flat gtb enclav nanad nagari, nu...",14754,0,"[167.0, 384.0, 712.0, 403.0, 439.5, 393.5]"
274,DELHI-110093,"(delhi-110093, text, 2337, 1653)",14754,0,"[168.0, 417.0, 323.0, 436.0, 245.5, 426.5]"


In [78]:
df.shape

(4754, 5)

In [79]:
# df[df['label'] == 2]
# df['groups']

In [80]:
X = df[['groups']]
X.head()

,groups
270,ACME VISUAL SYSTEMS
271,TAX INVOICE
272,ACME VISUAL SYSTEMS
273,"808, JANTA FLAT, GTB ENCLAVE, NANAD NAGARI"
274,DELHI-110093


In [81]:
y = df['label']

## Word tokenizer

In [82]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer

In [83]:
c_vectorizer = CountVectorizer()
h_vectorizer = HashingVectorizer()
f_vectorizer = TfidfVectorizer()

In [84]:
c_vectorizer.fit(df['groups'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [87]:
X = vectorizer.transform(df['groups'])

## Train test split

In [88]:
from sklearn.model_selection import train_test_split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)